<a href="https://colab.research.google.com/github/Gabbers18/Python_Projects/blob/main/Topic_Modeling_Articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1 Week 9 Topic Modeling**



In [ ]:
!pip install newsapi-python
!pip install git+https://github.com/rwalk/gsdmm.git

  Cloning https://github.com/rwalk/gsdmm.git to /tmp/pip-req-build-qhij0mp0
  Running command git clone --filter=blob:none --quiet https://github.com/rwalk/gsdmm.git /tmp/pip-req-build-qhij0mp0
  Resolved https://github.com/rwalk/gsdmm.git to commit 4ad1b6b6976743681ee4976b4573463d359214ee
  Preparing metadata (setup.py) ... done


In [ ]:
import re
import pandas as pd
import gensim
import numpy as np
from gsdmm import MovieGroupProcess
from textblob import TextBlob
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora.textcorpus import lower_to_unicode
from gensim import corpora, models
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
ps = nltk.porter.PorterStemmer()

from nltk.stem import WordNetLemmatizer

nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import unicodedata

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
from newsapi import NewsApiClient

# Init
# newsapi = NewsApiClient(api_key='insert your api key here')

In [ ]:
ng = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')


In [ ]:
ng.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


1. Randomly select 40% of the posts in the newsgroup dataset.

In [ ]:
# Referring to the documentation, I set the "seed" to 123 to make this sample reproducable.
ng_sample = ng.sample(frac = .40, random_state = 123)

In [ ]:
ng_sample.head()

,content,target,target_names
872,From: keith@cco.caltech.edu (Keith Allan Schne...,0,alt.atheism
10976,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,0,alt.atheism
2573,From: shaig@Think.COM (Shai Guday)\nSubject: R...,17,talk.politics.mideast
11214,From: maynard@ramsey.cs.laurentian.ca (Roger M...,10,rec.sport.hockey
10939,From: ak296@yfn.ysu.edu (John R. Daker)\nSubje...,7,rec.autos


2. Create descriptive statistics.

In [ ]:
# I found the number of total articles.
total_articles = ng_sample.shape[0]
total_articles

4526

In [ ]:
# I counted the number of articles within each group (target names).
# I referred to a past assignment to find how to do this.
articles_per_group = ng_sample['target_names'].value_counts()
articles_per_group

,count
target_names,
sci.crypt,270
soc.religion.christian,258
comp.windows.x,253
sci.electronics,247
rec.sport.baseball,242
comp.os.ms-windows.misc,241
comp.graphics,235
comp.sys.mac.hardware,234
rec.autos,233


3. Prepare the text for topic modeling.

In [ ]:
articles = []
for article in ng_sample['content'].tolist():
  if article:
    articles.append(article)

articles

["From: keith@cco.caltech.edu (Keith Allan Schneider)\nSubject: Re: Keith Schneider - Stealth Poster?\nOrganization: California Institute of Technology, Pasadena\nLines: 12\nNNTP-Posting-Host: punisher.caltech.edu\n\nsandvik@newton.apple.com (Kent Sandvik) writes:\n\n>>To borrow from philosophy, you don't truly understand the color red\n>>until you have seen it.\n>Not true, even if you have experienced the color red you still might\n>have a different interpretation of it.\n\nBut, you wouldn't know what red *was*, and you certainly couldn't judge\nit subjectively.  And, objectivity is not applicable, since you are wanting\nto discuss the merits of red.\n\nkeith\n",
 "From: livesey@solntze.wpd.sgi.com (Jon Livesey)\nSubject: Re: Slavery (was Re: Why is sex only allowed in marriage: ...)\nOrganization: sgi\nLines: 37\nDistribution: world\nNNTP-Posting-Host: solntze.wpd.sgi.com\n\nIn article <1993Apr15.081303.16532@Cadence.COM>, mas@Cadence.COM (Masud Khan) writes:\n|> \n|> Leonard, I'll g

In [ ]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts = sorted(cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]

        print("\nCluster %s , %s"%(cluster, sort_dicts))

In [ ]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
# I added more stopwords based on a series of outputs. Got this code from our class notebook.
stop = STOPWORDS.union(set(['...', 'say', 'will', 'know', 'first', '\\/', '>>', '*.','|>',');','as', '10', 'be', ').', '--', 'edu', 'com','re', '."']))

# How many topics would you like
# Expirement with this number
num1 = 10

In [ ]:
#Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings
def text_preprocessing(text):

  #makes all text lowercase
  text = text.lower()

  if True:
    text = text.split()
    new_text = []
    for word in text:
      if word not in stop:
        if word in contractions:
          new_text.append(contractions[word])
        else:
          new_text.append(word)
  text = " ".join(new_text)

  #formats words and removes unwanted characters
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r'\'', ' ', text)
  text = re.sub(r'\'', ' ', text)

  #Tokenize each word
  text = nltk.WordPunctTokenizer().tokenize(text)
  text = [word for word in text if word not in stop] # Removing Stop words

  #Lemmatize each word
  text = [nltk.stem.WordNetLemmatizer().lemmatize(token, pos='v') for token in text if len(token)>1]

  return text

#convert list to string
def to_string(text):
  text = ' '.join(map(str, text))
  return text

In [ ]:
docs = []
for article in articles:
  docs.append(text_preprocessing(article))

In [ ]:
len(docs)

4526

In [ ]:
# pulled from our class notebook. gives each word its own unique integer ID.
dictionary = gensim.corpora.Dictionary(docs)


dictionary.filter_extremes(no_below=15, no_above=0.3, keep_n=100000)
vocab_length = len(dictionary)

bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

gsdmm = MovieGroupProcess(K= num1, alpha=0.1, beta=0.9, n_iters=15)


y = gsdmm.fit(docs, vocab_length)

#let us print using the function above
doc_count = np.array(gsdmm.cluster_doc_count)
print('Number of documents per topic :', doc_count)
top_index = doc_count.argsort()[-15:][::-1]
print('Most important clusters (by number of docs inside):', top_index)
top_words(gsdmm.cluster_word_distribution, top_index, 20)

In stage 0: transferred 4015 clusters with 10 clusters populated
In stage 1: transferred 867 clusters with 10 clusters populated
In stage 2: transferred 371 clusters with 10 clusters populated
In stage 3: transferred 207 clusters with 10 clusters populated
In stage 4: transferred 114 clusters with 10 clusters populated
In stage 5: transferred 123 clusters with 10 clusters populated
In stage 6: transferred 113 clusters with 10 clusters populated
In stage 7: transferred 93 clusters with 10 clusters populated
In stage 8: transferred 99 clusters with 10 clusters populated
In stage 9: transferred 112 clusters with 10 clusters populated
In stage 10: transferred 87 clusters with 10 clusters populated
In stage 11: transferred 87 clusters with 10 clusters populated
In stage 12: transferred 87 clusters with 10 clusters populated
In stage 13: transferred 72 clusters with 10 clusters populated
In stage 14: transferred 76 clusters with 10 clusters populated
Number of documents per topic : [ 155  20

4. Create topic model using LDA

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
# Make an LDA model.
lda_model = models.LdaModel(corpus=bow_corpus, id2word=dictionary, num_topics=5, random_state=42, passes=10)

# Display topics
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.006*"key" + 0.004*"need" + 0.004*"mail" + 0.004*"file" + 0.004*"make" + 0.004*"work" + 0.004*"chip" + 0.004*"may" + 0.004*"program" + 0.004*"new"')
(1, '0.009*"think" + 0.009*"people" + 0.007*"god" + 0.006*"make" + 0.006*"say" + 0.006*"see" + 0.005*"believe" + 0.005*"go" + 0.005*"time" + 0.004*"jesus"')
(2, '0.009*"00" + 0.008*"game" + 0.006*"play" + 0.006*"team" + 0.006*"ca" + 0.006*"50" + 0.005*"15" + 0.005*"20" + 0.005*"12" + 0.005*"40"')
(3, '0.213*"ax" + 0.016*"max" + 0.010*"windows" + 0.007*"scsi" + 0.007*"card" + 0.007*"drive" + 0.006*"window" + 0.005*"work" + 0.005*"pc" + 0.005*"mac"')
(4, '0.009*"gun" + 0.007*"people" + 0.006*"go" + 0.006*"say" + 0.004*"state" + 0.004*"space" + 0.004*"us" + 0.004*"take" + 0.004*"time" + 0.003*"government"')


5. Interpret the model.

Here, I have pulled 5 key topics from our articles.
- Topic 1, listed as 0, might be discussing a work environment, with repeated metions of "work," "mail," and "file," maybe emailing files or sharing between coworkers?
- Topic 2, listed as 1, appears to discuss the Christain religion. This can be inferred from the repeated mention of specific words such as "god," "jesus," "believe."
- Topic 3, listed as 2, appears to discuss a sports game of some sort, because or repeated metnions of "game," "play," and numbers which might represent score of the game.
- Topic 4, listed as 3, appears to discuss computers or their technologies, which can be inferred from repeated metions of "card," "drive," "pc," "mac," etc. Potentailly comparing the hardware of pc and macs?
- Topic 5, listed as 4, appears to disuss politics or laws/rights, based on the repeated mention of "gun," "state," and "government."
